Загружаем библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

Загружаем данные

In [2]:
clicks_df = pd.read_csv('data/yoochoose-clicks.dat', sep=',', header=None,
                        names=['session_id', 'timestamp', 'item_id', 'category'])

buys_df = pd.read_csv('data/yoochoose-buys.dat', sep=',', header=None,
                      names=['session_id', 'timestamp', 'item_id', 'price', 'quantity'])

/var/folders/l2/mh6902w91rxdry1pssd9vxlw0000gn/T/ipykernel_29593/2791895665.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  clicks_df = pd.read_csv('data/yoochoose-clicks.dat', sep=',', header=None,


Посмотрим на основные характеристики датафреймов

In [4]:
print(clicks_df.shape, buys_df.shape) # размер датафреймов

(33003944, 4) (1150753, 5)


In [5]:
print("Пример строк из clicks_df:")
print(clicks_df.head())

print("\nИнформация о clicks_df:")
print(clicks_df.info())

print("\nПример строк из buys_df:")
print(buys_df.head())

print("\nИнформация о buys_df:")
print(buys_df.info())

Пример строк из clicks_df:
   session_id                 timestamp    item_id category
0           1  2014-04-07T10:51:09.277Z  214536502        0
1           1  2014-04-07T10:54:09.868Z  214536500        0
2           1  2014-04-07T10:54:46.998Z  214536506        0
3           1  2014-04-07T10:57:00.306Z  214577561        0
4           2  2014-04-07T13:56:37.614Z  214662742        0

Информация о clicks_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33003944 entries, 0 to 33003943
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   session_id  int64 
 1   timestamp   object
 2   item_id     int64 
 3   category    object
dtypes: int64(2), object(2)
memory usage: 1007.2+ MB
None

Пример строк из buys_df:
   session_id                 timestamp    item_id  price  quantity
0      420374  2014-04-06T18:44:58.314Z  214537888  12462         1
1      420374  2014-04-06T18:44:58.325Z  214537850  10471         1
2      281626  2014-04-06T09:40:13.032Z  2

Проверка наличия артефактов в данных

In [6]:
clicks_df.category.value_counts()

category
0             14548992
S             10769610
0              1788661
1              1671754
2              1292249
                ...   
2089179505           1
2089318108           1
193464283            1
2089028904           1
2089314263           1
Name: count, Length: 340, dtype: int64

Как можно заметить, некоторые значения дублируются, хотя семантически обозначают одно и то же (0 и "0" например). Приведем значения столбца category к единому формату _str_

In [3]:
clicks_df['category'] = clicks_df['category'].astype(str)
clicks_df['category'] = clicks_df['category'].str.strip()
print(clicks_df['category'].value_counts())

category
0             16337653
S             10769610
1              1671754
2              1292249
3               789713
                ...   
2089179505           1
2089318108           1
193464283            1
2089028904           1
2089314263           1
Name: count, Length: 339, dtype: int64


In [4]:
clicks_df.shape

(33003944, 4)

Проверка пропусков и дубликатов

In [8]:
print("\nПропуски в clicks_df:")
print(clicks_df.isnull().sum())

print("\nПропуски в buys_df:")
print(buys_df.isnull().sum())

# Проверка дубликатов
clicks_duplicates = clicks_df.duplicated().sum()
buys_duplicates = buys_df.duplicated().sum()

print(f"\nДубликаты в clicks_df: {clicks_duplicates}")
print(f"Дубликаты в buys_df: {buys_duplicates}")


Пропуски в clicks_df:
session_id    0
timestamp     0
item_id       0
category      0
dtype: int64

Пропуски в buys_df:
session_id    0
timestamp     0
item_id       0
price         0
quantity      0
dtype: int64

Дубликаты в clicks_df: 68
Дубликаты в buys_df: 146


* Пропуски в данных отсутствуют
* Убедимся на примере датафрейма _clicks_df_, что дубликаты действительно существуют

In [9]:
dups_clicks = clicks_df[clicks_df.duplicated(keep=False)]
print(dups_clicks.sort_values(by=['session_id','timestamp','item_id','category']).head(50))

Удалим дубликаты

In [5]:
clicks_df = clicks_df.drop_duplicates()
buys_df = buys_df.drop_duplicates()

In [6]:
clicks_df.shape

(33003876, 4)

In [7]:
clicks_duplicates = clicks_df.duplicated().sum()
buys_duplicates = buys_df.duplicated().sum()

In [8]:
print(clicks_duplicates, buys_duplicates)

0 0


Преобразуем тип временной метки в __datetime__ и сортируем события в сессии по времени, чтобы модели, учитывающие последовательность кликов (например, GRU4Rec), могли корректно работать

In [9]:
clicks_df['timestamp'] = pd.to_datetime(clicks_df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%fZ')
buys_df['timestamp'] = pd.to_datetime(buys_df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%fZ')

clicks_df = clicks_df.sort_values(by=['session_id', 'timestamp'])
buys_df = buys_df.sort_values(by=['session_id', 'timestamp'])

Сохраним обработанные датафреймы для дальнейшего анализа

In [ ]:
clicks_df.to_csv('clicks_processed.csv', index=False)
buys_df.to_csv('buys_processed.csv', index=False)